In [341]:
# !python.exe -m pip install --upgrade pip
# !pip3 install pigar -q
import pigar

In [344]:
!pigar generate
# !pip3 install -r requirements.txt -q

^C


In [343]:
import numpy as np
import pandas as pd
import torch

import matplotlib.plotlib as plt
import seaborn

import pathlib as pl

ModuleNotFoundError: No module named 'matplotlib'

In [345]:
DFN = 'data'

In [346]:
def load(csv_fn: str, location: pl.Path =pl.Path('./').joinpath(DFN), **pdkwargs) -> pd.DataFrame:
    """loads CSV file to pandas DataFrame if it exists"""
    try:
        with open(fp := location.joinpath(csv_fn), 'r') as f:
            result = pd.read_csv(f, **pdkwargs)
            print(f"{fp.stat().st_size / 2 ** 20:.3f}MB csv file {fp} has been loaded successfully")
        return result
    except FileNotFoundError:
        print(f"file {csv_fn} doesn't exist")
        return None

In [347]:
train_df = load('problem_train.csv', low_memory=False)
# test_df = load('problem_test.csv', low_memory=False)
labels_df = load('problem_labels.csv')

14.085MB csv file data\problem_train.csv has been loaded successfully
0.263MB csv file data\problem_labels.csv has been loaded successfully


In [348]:
# everything (except auto-indexing as object) is treated as int64
labels_df.dtypes.unique(), labels_df.shape, labels_df.select_dtypes(include=['int64']).columns.size

(array([dtype('int64')], dtype=object), (8000, 15), 15)

In [349]:
# we've inferred text, float and integer dtypes from train data
train_df.dtypes.unique()
# type O means strings, pandas stores pointers to strings (python objects) instead of strings themselves 
train_df.select_dtypes(include=['O']).columns.size, train_df.select_dtypes(include=['float64']).columns.size, train_df.select_dtypes(include=['int64']).columns.size

(1025, 345, 9)

In [350]:
sum((1025, 345, 9)) == train_df.columns.size

True

In [351]:
train_df.select_dtypes(include=['int64']).describe()

,id,n_0047,n_0050,n_0052,n_0061,n_0075,n_0091,o_0176,o_0264
count,8000.000000,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.000000,8000.000000
mean,9220.303375,1.0,1.0,1.0,1.0,1.0,1.0,198.395500,4.585875
std,5236.897900,0.0,0.0,0.0,0.0,0.0,0.0,118.487761,2.237912
min,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000
25%,4773.500000,1.0,1.0,1.0,1.0,1.0,1.0,92.000000,4.000000
50%,9221.500000,1.0,1.0,1.0,1.0,1.0,1.0,193.000000,5.000000
75%,13702.750000,1.0,1.0,1.0,1.0,1.0,1.0,315.000000,6.000000
max,18302.000000,1.0,1.0,1.0,1.0,1.0,1.0,369.000000,10.000000


In [352]:
train_df.select_dtypes(include=['float64']).describe()

,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,n_0008,n_0009,...,c_1072,c_1129,c_1148,c_1168,c_1230,c_1257,c_1277,c_1281,c_1308,c_1332
count,12.000000,388.000000,7662.000000,112.000000,58.000000,7658.000000,1348.000000,995.000000,5.000000,34.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.307692,0.655744,0.040935,0.102749,0.352874,0.395981,0.193175,0.012012,0.422222,0.129241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.245454,0.216399,0.024599,0.153007,0.209337,0.085000,0.236683,0.041382,0.395967,0.162008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.134615,0.428571,0.031974,0.011905,0.200000,0.342105,0.000000,0.000000,0.111111,0.015560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.307692,0.702381,0.041369,0.047619,0.283333,0.394737,0.200000,0.000000,0.444444,0.049793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.403846,0.857143,0.048950,0.142857,0.525000,0.421053,0.400000,0.011905,0.555556,0.186722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,0.846154,1.000000,1.000000,1.000000,0.833333,1.000000,1.000000,1.000000,1.000000,0.597510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [460]:
strings_part = train_df.select_dtypes(include=['O'])
string_stats = strings_part.describe()
string_stats

,release,c_0327,c_0328,c_0329,c_0330,c_0331,c_0332,c_0333,c_0334,c_0335,...,c_1368,c_1369,c_1370,c_1371,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
count,8000,22,512,5,157,6,1690,482,26,821,...,3,48,377,2,7998,440,3155,563,7,1908
unique,3,2,5,4,6,3,2,2,9,5,...,3,11,3,2,2,5,24,19,2,2
top,a,a,d,e,c,c,a,b,i,b,...,d,m,b,c,a,c,b,j,b,b
freq,3838,12,248,2,68,4,1498,383,15,468,...,1,22,312,1,7460,167,1559,232,4,1441


In [461]:
string_stats.loc['unique'].sort_values(ascending=False).head(5)

c_1259    32
c_1374    24
c_0965    23
c_0523    23
c_0434    23
Name: unique, dtype: object

In [443]:
train_df['c_1259'].unique()

array(['n', 'e', 'w', 'm', 'p', 's', 'u', 'k', 'j', 'b', 'i', 'B', 'q',
       'a', 'h', 'G', 'c', 'd', 'C', 'v', 'y', 'D', 'o', 'l', 'A', 'z',
       't', 'r', 'F', 'x', 'g', 'f'], dtype=object)

In [444]:
from functools import reduce
# establish top K columns (by amount of unique strings) in dataframe
df_string_sample = string_stats.loc['unique'].sort_values(ascending=False).head(10).index
# extract unique values
df_string_sample_cols = [train_df[cn].dropna().unique() for cn in df_string_sample]
#  compare find intersection
reduce(np.intersect1d, df_string_sample_cols) # ~ map and accumulate

array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'u', 'v', 'w'], dtype=object)

In [486]:
# list all possible unique values
pd.unique(strings_part.values.ravel('K'))

array(['a', 'c', 'b', nan, 'd', 'e', 'f', 'i', 'h', 'k', 'm', 'g', 'r',
       's', 'o', 'j', 'l', 'q', 'p', 't', 'n', 'w', 'u', 'v', 'x', 'y',
       'B', 'G', 'C', 'D', 'A', 'z', 'F'], dtype=object)

String data consists of 33 different (mostly lowercase) latin letters, the only unique string value amongst top-50 columns is "**p**"

that doesn't make any sense i.e. could be one-hot-encoded (columnwise) as it doesn't bring any semantic meaning